# This code is for testing both agents at petting zoo enviroment

# Important: When we make predictions from model we get 2 values, first one is action.

In [7]:
 !pip install swig
 !pip install box2d-py
 !pip install gymnasium
 !pip install "gymnasium[atari,accept-rom-license]"
 !pip install "stable-baselines3[extra]"
 !pip install "pettingzoo[all]"
 !pip install supersuit
 !pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp310-cp310-linux_x86_64.whl size=2376417 sha256=e226d5fbc5a3a9f99e38a4dcbeae5e75a69e841a5d946067d8c8944d43e65131
  Stored in directory: /root/.cache/pip/wheels/47/01/d2/6a780da77ccb98b1d2facdd520a8d10838a03b590f6f8d50c0
Successfully built box2d-py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.3 MB/s eta 0:00:00
  Using cached multi_agent_ale_py-0.1.11-cp310-cp310-linux_x86_64.whl
  Using cached pygame-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached chess-1.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached rlcard-1.0.5-py3-none-any.whl
  Using cached Shimmy-2.0.0-py3-none-any.whl.metadata (3.5 k

In [9]:
!pip install "autorom[accept-rom-license]"
!AutoROM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446667 sha256=69c424ccc3161926a117fe098a5b7fc5734362839c120d0e16087684e0f147d8
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms
	/usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.

I own a license to these Atari 2600 ROMs.
I agree to not distribute these ROMs and wish to proceed: [Y/n]: Y
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/py

In [ ]:
path_model1_left="../../models/pong/left_best_model.zip"
path_model2_right="../../models/pong/right_best_model.zip"

In [26]:
import numpy as np
import supersuit as ss
from pettingzoo.atari import pong_v3
from PIL import Image
import imageio
from stable_baselines3 import PPO


env = pong_v3.env(render_mode="rgb_array")
env.reset(seed=42)

# Pre-process using SuperSuit
env = ss.color_reduction_v0(env, mode="B")
env = ss.resize_v1(env, x_size=84, y_size=84)
env = ss.frame_stack_v1(env, 4)
env = ss.dtype_v0(env, dtype=np.float32)
env = ss.normalize_obs_v0(env, env_min=0, env_max=1)

env.reset()
observation, reward, termination, truncation, info = env.last()
print("Obs. shape is {}".format(observation.shape))
print("     min-max are {}-{}".format(np.min(observation), np.max(observation)))

# Record frames for video
frames = []


# Load the agents
model1 = PPO.load(path_model1_left) # left paddle model
model2 = PPO.load(path_model2_right)  # Right paddle model
rewards = {agent: 0 for agent in env.possible_agents}

# We evaluate here using an AEC environments
env.reset(seed=1234)

for agent in env.agent_iter():
    obs, reward, termination, truncation, info = env.last()

    for a in env.agents:
        rewards[a] += env.rewards[a]

    if termination or truncation:
        break
    else:
        obs = obs.transpose(2, 0, 1)  # Channel-first format
        if agent == env.possible_agents[0]:
            action, _ = model1.predict(obs)
        else:
            action, _ = model2.predict(obs)

    env.step(action)

    # Capture the RGB frame
    rendered_frame = env.render()
    frames.append(rendered_frame)

env.close()

# print reward
for agent in env.possible_agents:
    print("Agent '{}' reward: {}".format(agent, rewards[agent]))

# export video
# filename = "exports/" + "pong.gif"
filename = "pong.gif"
imageio.mimwrite(filename, frames, duration=20)
print(">>> Video exported at '{}'!".format(filename))

Obs. shape is (84, 84, 4)
     min-max are 0.0-0.9254902005195618
Agent 'first_0' reward: -16
Agent 'second_0' reward: 16
>>> Video exported at 'pong.gif'!
